# *Image Denoising* using Bivariate Monotone Regression^[My text editor, Cursor, used Gemini 2.0 Flash-Thinking to generate most of this code.]

In [ ]:
library(CVXR)

monotone_denoise_2d = function(noisy_image) {
  rows = nrow(noisy_image)
  cols = ncol(noisy_image)

  # Create the variable for the denoised image
  denoised_image = Variable(dim(noisy_image))

  # Define the objective function: minimize the sum of squared differences
  objective = Minimize(sum_squares(denoised_image - noisy_image))

  # Define the constraints: monotonicity
  constraints = list()
  for (r in 1:rows) {
    for (c in 1:(cols - 1)) {
      constraints = c(constraints, denoised_image[r, c] <= denoised_image[r, c + 1])
    }
  }
  for (r in 1:(rows - 1)) {
    for (c in 1:cols) {
      constraints = c(constraints, denoised_image[r, c] <= denoised_image[r + 1, c])
    }
  }

  # Formulate the problem
  problem = Problem(objective, constraints)

  # Solve the problem
  result = solve(problem)
  
  result$getValue(denoised_image)
}

noisy_image = matrix(runif(30*30), nrow=30) + outer(seq(0, 1, length.out = 30), rep(0.5, 30))
denoised_image_np = monotone_denoise_2d(noisy_image)

In [ ]:
show.image = function(image) {
  df = as.data.frame.table(image)
  colnames(df) = c("row", "col", "value")
  ggplot(df, aes(x = col, y = row, fill = value)) +
    geom_raster() +
    scale_fill_gradient(low = "black", high = "white") +
    coord_equal() + guides(fill=FALSE)
}

show.image(noisy_image)
show.image(denoised_image_np)

In [ ]:
library(jpeg)
image_url = "https://upload.wikimedia.org/wikipedia/commons/thumb/b/b4/The_Sun_by_the_Atmospheric_Imaging_Assembly_of_NASA%27s_Solar_Dynamics_Observatory_-_20100819.jpg/640px-The_Sun_by_the_Atmospheric_Imaging_Assembly_of_NASA%27s_Solar_Dynamics_Observatory_-_20100819.jpg"
download.file(image_url, "sun.jpg", mode = "wb")
img = readJPEG("sun.jpg")

image.to.gray = function(image) {
  if (length(dim(image)) == 3) {
    return(0.299 * image[, , 1] + 0.587 * image[, , 2] + 0.114 * image[, , 3])
  } else {
    return(image)
  }
}
gray_img = image.to.gray(img)
show.image(gray_img)

resize_image = function(image_array, target_rows, target_cols) {
  current_rows = dim(image_array)[1]
  current_cols = dim(image_array)[2]

  resized_image = matrix(0, nrow = target_rows, ncol = target_cols)

  row_ratio = current_rows / target_rows
  col_ratio = current_cols / target_cols

  for (r in 1:target_rows) {
    for (c in 1:target_cols) {
      start_row = floor((r - 1) * row_ratio) + 1
      end_row = min(current_rows, floor(r * row_ratio))
      start_col = floor((c - 1) * col_ratio) + 1
      end_col = min(current_cols, floor(c * col_ratio))

      resized_image[r, c] = mean(image_array[start_row:end_row, start_col:end_col])
    }
  }
  return(resized_image)
}

# Downsample the image to 30x30
downsampled_img = resize_image(gray_img, 30, 30)

# Denoise the downsampled image
denoised_img_np = monotone_denoise_2d(downsampled_img)

show.image(downsampled_img)
show.image(denoised_img_np)